In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
from skimage.transform import resize
import random
from segmentation_mask_overlay import overlay_masks
import matplotlib.colors as mcolors

ModuleNotFoundError: No module named 'segmentation_mask_overlay'

In [3]:
path_ACDC_processed = '/Users/poojakishore/Downloads/ACDC/'
ACDC_ORIG_DATA = '/Users/poojakishore/Downloads/FCT-Pytorch-main/ACDC' # change it to the downloaded ACDC data
ACDC_Save = './DataProcessed/' #change it to ACDC when you are running it
os.makedirs(ACDC_Save)

## Getting data from actual ACDC dataset Training

In [3]:
folders = os.listdir(ACDC_ORIG_DATA+'/training')

In [4]:
folders.remove('.DS_Store')

In [5]:
# Read cfg file in each of the folders
patientDetails=[]
for folder in folders:
    cfg_file = os.path.join(ACDC_ORIG_DATA+'/training', folder, 'Info.cfg')
    config_dict = {}
    with open(cfg_file, 'r') as file:
        lines = file.readlines()
        id = folder.split('patient')[-1]
        config_dict['patientID'] = id
        for line in lines:
            key, value = line.strip().split(': ')
            # Strip any extra spaces around the key and value
            key = key.strip()
            value = value.strip()
            # Add key-value pair to dictionary
            config_dict[key] = value
    patientDetails.append(config_dict)

In [6]:
patientDetails

[{'patientID': '015',
  'ED': '1',
  'ES': '10',
  'Group': 'DCM',
  'Height': '158.0',
  'NbFrame': '21',
  'Weight': '57.0'},
 {'patientID': '012',
  'ED': '1',
  'ES': '13',
  'Group': 'DCM',
  'Height': '160.0',
  'NbFrame': '30',
  'Weight': '59.0'},
 {'patientID': '024',
  'ED': '1',
  'ES': '9',
  'Group': 'HCM',
  'Height': '175.0',
  'NbFrame': '28',
  'Weight': '85.0'},
 {'patientID': '023',
  'ED': '1',
  'ES': '9',
  'Group': 'HCM',
  'Height': '166.0',
  'NbFrame': '25',
  'Weight': '74.0'},
 {'patientID': '048',
  'ED': '1',
  'ES': '8',
  'Group': 'MINF',
  'Height': '167.0',
  'NbFrame': '28',
  'Weight': '73.0'},
 {'patientID': '083',
  'ED': '1',
  'ES': '8',
  'Group': 'RV',
  'Height': '160.0',
  'NbFrame': '17',
  'Weight': '58.0'},
 {'patientID': '077',
  'ED': '1',
  'ES': '9',
  'Group': 'NOR',
  'Height': '180',
  'NbFrame': '25',
  'Weight': '75'},
 {'patientID': '070',
  'ED': '1',
  'ES': '10',
  'Group': 'NOR',
  'Height': '158.0',
  'NbFrame': '28',
  'Wei

In [7]:
#separate the patients based on group
groupDCM = [patient for patient in patientDetails if patient['Group'] == 'DCM']
groupMINF = [patient for patient in patientDetails if patient['Group'] == 'MINF']
groupNOR = [patient for patient in patientDetails if patient['Group'] == 'NOR']
groupRV = [patient for patient in patientDetails if patient['Group'] == 'RV']
groupHCM = [patient for patient in patientDetails if patient['Group'] == 'HCM']

In [8]:
#pick 18 random patients from each group
random.seed(42)

groupDCM_train = random.sample(groupDCM, 18)
groupMINF_train = random.sample(groupMINF, 18)
groupNOR_train = random.sample(groupNOR, 18)
groupRV_train = random.sample(groupRV, 18)
groupHCM_train = random.sample(groupHCM, 18)

In [10]:
#combine all the train data
trainDataPreprocess = np.concatenate((groupDCM_train,groupMINF_train,groupNOR_train,groupRV_train,groupHCM_train))
# print(trainDataPreprocess)

for patient in trainDataPreprocess:
    if patient['patientID'] == '140':
        print(patient['patientID'])

In [11]:
groupDCM_valid = random.sample([patient for patient in groupDCM if patient not in groupDCM_train],2)
groupMINF_valid = random.sample([patient for patient in groupMINF if patient not in groupMINF_train],2)
groupNOR_valid = random.sample([patient for patient in groupNOR if patient not in groupNOR_train],2)
groupRV_valid = random.sample([patient for patient in groupRV if patient not in groupRV_train],2)
groupHCM_valid = random.sample([patient for patient in groupHCM if patient not in groupHCM_train],2)

In [12]:
def change_frame_cha(groupData):
    for item in groupData:
        if len(item['ED']) == 1:  # Check if 'ED' value is a single digit
            item['ED'] = item['ED'].zfill(2)  # Pad with leading zero to make it two digits
        if len(item['ES']) == 1:  # Check if 'ES' value is a single digit
            item['ES'] = item['ES'].zfill(2)  # Pad with leading zero to make it two digits
    return groupData

In [13]:
groupDCM_train = change_frame_cha(groupDCM_train)
groupMINF_train = change_frame_cha(groupMINF_train)
groupNOR_train = change_frame_cha(groupNOR_train)
groupRV_train = change_frame_cha(groupRV_train)
groupHCM_train = change_frame_cha(groupHCM_train)

#Fix Valid data
groupDCM_valid = change_frame_cha(groupDCM_valid)
groupMINF_valid = change_frame_cha(groupMINF_valid)
groupNOR_valid = change_frame_cha(groupNOR_valid)
groupRV_valid = change_frame_cha(groupRV_valid)
groupHCM_valid = change_frame_cha(groupHCM_valid)

### Save train data

In [16]:
trainDataPreprocess = np.concatenate((groupDCM_train,groupMINF_train,groupNOR_train,groupRV_train,groupHCM_train))
# print(trainDataPreprocess)
# 
# for patient in trainDataPreprocess:
#     if patient['patientID'] == '140':
#         print(patient['patientID'])

In [15]:
def save_one_frame(patientID,frameNum,savePath, size):
    imagePath = ACDC_ORIG_DATA+'/training/' +'patient'+ patientID + '/patient'+ patientID + '_frame' + frameNum + '.nii.gz'
    maskPath = ACDC_ORIG_DATA+'/training/' +'patient'+patientID + '/patient'+ patientID +'_frame'+ frameNum + '_gt.nii.gz'

    #load image and mask
    image = nib.load(imagePath)
    image_data = image.get_fdata()
    mask = nib.load(maskPath)
    mask_data = mask.get_fdata()

    # Resize and normalize each frame and save it
    # filePathName=[]
    for i in range(image_data.shape[2]):
        resized_image_slice = resize(image_data[:, :, i], (size, size), order=0, preserve_range=True) #resize(image_data[:, :, i], (size, size), anti_aliasing=True)
        resized_mask_slice = resize(mask_data[:, :, i], (size, size), order=0, preserve_range=True) #resize(mask_data[:, :, i], (size, size), anti_aliasing=True)
        # Normalize the image slice to range [-1, 1]
        min_val = np.min(resized_image_slice)
        max_val = np.max(resized_image_slice)

        normalized_image_slice = 2 * (resized_image_slice - min_val) / (max_val - min_val) - 1
        print(normalized_image_slice.shape)
        fileName = savePath + f'_{i}'
        print(fileName)
        # filePathName.append(fileName)
        np.savez_compressed(fileName, img=normalized_image_slice, label=resized_mask_slice)
    print("successfully saved the files for patientID: ", patientID)
    # return filePathName

In [18]:
# trainFileNames= []
for patient in trainDataPreprocess:
    #save ed frame
    edFilePath=ACDC_Save+'train/case_'+ patient["patientID"] +'_sliceED'
    fileNamesEd = save_one_frame(patient['patientID'],patient['ED'],edFilePath,224)
    
    #save es frame
    esFilePath=ACDC_Save+ 'train/'+ 'case_'+ patient["patientID"] +'_sliceES'
    fileNamesEs=save_one_frame(patient['patientID'],patient['ES'],esFilePath,224)
    

(224, 224)
./DataProcessed/train/case_014_sliceED_0
(224, 224)
./DataProcessed/train/case_014_sliceED_1
(224, 224)
./DataProcessed/train/case_014_sliceED_2
(224, 224)
./DataProcessed/train/case_014_sliceED_3
(224, 224)
./DataProcessed/train/case_014_sliceED_4
(224, 224)
./DataProcessed/train/case_014_sliceED_5
(224, 224)
./DataProcessed/train/case_014_sliceED_6
(224, 224)
./DataProcessed/train/case_014_sliceED_7
(224, 224)
./DataProcessed/train/case_014_sliceED_8
(224, 224)
./DataProcessed/train/case_014_sliceED_9
successfully saved the files for patientID:  014
(224, 224)
./DataProcessed/train/case_014_sliceES_0
(224, 224)
./DataProcessed/train/case_014_sliceES_1
(224, 224)
./DataProcessed/train/case_014_sliceES_2
(224, 224)
./DataProcessed/train/case_014_sliceES_3
(224, 224)
./DataProcessed/train/case_014_sliceES_4
(224, 224)
./DataProcessed/train/case_014_sliceES_5
(224, 224)
./DataProcessed/train/case_014_sliceES_6
(224, 224)
./DataProcessed/train/case_014_sliceES_7
(224, 224)
./Da

In [19]:
def read_file_names(path):
    fileNames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".npz"):
                fileNames.append(os.path.join(root, file))
    return fileNames

In [27]:
trainFileNames_new= read_file_names(ACDC_Save+'train')
trainFileNames_modified = np.char.replace(trainFileNames_new, ACDC_Save+'train/', '')

In [28]:
trainFileNames_modified

array(['case_043_sliceES_10.npz', 'case_060_sliceES_7.npz',
       'case_096_sliceED_13.npz', ..., 'case_074_sliceED_2.npz',
       'case_013_sliceED_1.npz', 'case_093_sliceED_0.npz'], dtype='<U23')

In [29]:
# save the array to a file as txt file
np.savetxt(ACDC_Save+'lists_ACDC/train.txt', trainFileNames_modified, fmt='%s')

### Save validation data

In [24]:
validDataPreprocessing = np.concatenate((groupDCM_valid,groupMINF_valid,groupNOR_valid,groupRV_valid,groupHCM_valid))

for patient in validDataPreprocessing:
    #save ed frame
    edFilePath=ACDC_Save+ 'valid/case_'+ patient["patientID"] +'_sliceED'
    save_one_frame(patient['patientID'],patient['ED'],edFilePath,224)
    #save es frame
    esFilePath=ACDC_Save+'valid/'+ 'case_'+ patient["patientID"] +'_sliceES'
    save_one_frame(patient['patientID'],patient['ES'],esFilePath,224)

(224, 224)
./DataProcessed/valid/case_007_sliceED_0
(224, 224)
./DataProcessed/valid/case_007_sliceED_1
(224, 224)
./DataProcessed/valid/case_007_sliceED_2
(224, 224)
./DataProcessed/valid/case_007_sliceED_3
(224, 224)
./DataProcessed/valid/case_007_sliceED_4
(224, 224)
./DataProcessed/valid/case_007_sliceED_5
(224, 224)
./DataProcessed/valid/case_007_sliceED_6
(224, 224)
./DataProcessed/valid/case_007_sliceED_7
(224, 224)
./DataProcessed/valid/case_007_sliceED_8
(224, 224)
./DataProcessed/valid/case_007_sliceED_9
successfully saved the files for patientID:  007
(224, 224)
./DataProcessed/valid/case_007_sliceES_0
(224, 224)
./DataProcessed/valid/case_007_sliceES_1
(224, 224)
./DataProcessed/valid/case_007_sliceES_2
(224, 224)
./DataProcessed/valid/case_007_sliceES_3
(224, 224)
./DataProcessed/valid/case_007_sliceES_4
(224, 224)
./DataProcessed/valid/case_007_sliceES_5
(224, 224)
./DataProcessed/valid/case_007_sliceES_6
(224, 224)
./DataProcessed/valid/case_007_sliceES_7
(224, 224)
./Da

In [30]:
validFileNames_new= read_file_names(ACDC_Save+'valid')
validFileNames_modified = np.char.replace(validFileNames_new, ACDC_Save+'valid/', '')
np.savetxt(ACDC_Save+'lists_ACDC/valid.txt', validFileNames_modified, fmt='%s')

In [31]:
validFileNames_modified

array(['case_007_sliceES_4.npz', 'case_007_sliceED_9.npz',
       'case_083_sliceES_4.npz', 'case_002_sliceED_6.npz',
       'case_072_sliceED_5.npz', 'case_029_sliceES_4.npz',
       'case_029_sliceED_9.npz', 'case_073_sliceES_6.npz',
       'case_029_sliceED_8.npz', 'case_029_sliceES_5.npz',
       'case_072_sliceED_4.npz', 'case_002_sliceED_7.npz',
       'case_083_sliceES_5.npz', 'case_007_sliceED_8.npz',
       'case_007_sliceES_5.npz', 'case_007_sliceES_7.npz',
       'case_002_sliceES_8.npz', 'case_002_sliceED_5.npz',
       'case_072_sliceED_6.npz', 'case_029_sliceES_7.npz',
       'case_073_sliceES_5.npz', 'case_073_sliceES_4.npz',
       'case_029_sliceES_6.npz', 'case_072_sliceED_7.npz',
       'case_002_sliceED_4.npz', 'case_002_sliceES_9.npz',
       'case_007_sliceES_6.npz', 'case_007_sliceES_2.npz',
       'case_029_sliceES_10.npz', 'case_083_sliceES_2.npz',
       'case_002_sliceED_0.npz', 'case_072_sliceED_3.npz',
       'case_073_sliceES_0.npz', 'case_029_sliceES_2.np

## Load data from ACDC test dataset

In [32]:
folders_test = os.listdir(ACDC_ORIG_DATA+'/testing')
print(folders_test)
folders_test.remove('.DS_Store')

['patient104', 'patient103', 'patient135', '.DS_Store', 'patient132', 'patient150', 'patient133', 'patient134', 'patient102', 'patient105', 'patient145', 'patient142', 'patient118', 'patient127', 'patient120', 'patient129', 'patient116', 'patient111', 'patient143', 'patient144', 'patient110', 'patient128', 'patient117', 'patient121', 'patient119', 'patient126', 'patient107', 'patient138', 'patient131', 'patient136', 'patient109', 'patient137', 'patient108', 'patient130', 'patient106', 'patient139', 'patient101', 'patient123', 'patient124', 'patient112', 'patient115', 'patient141', 'patient146', 'patient148', 'patient114', 'patient113', 'patient125', 'patient122', 'patient149', 'patient147', 'patient140']


In [35]:
#sort the array 
folders_test.sort()
folders_test

['patient101',
 'patient102',
 'patient103',
 'patient104',
 'patient105',
 'patient106',
 'patient107',
 'patient108',
 'patient109',
 'patient110',
 'patient111',
 'patient112',
 'patient113',
 'patient114',
 'patient115',
 'patient116',
 'patient117',
 'patient118',
 'patient119',
 'patient120',
 'patient121',
 'patient122',
 'patient123',
 'patient124',
 'patient125',
 'patient126',
 'patient127',
 'patient128',
 'patient129',
 'patient130',
 'patient131',
 'patient132',
 'patient133',
 'patient134',
 'patient135',
 'patient136',
 'patient137',
 'patient138',
 'patient139',
 'patient140',
 'patient141',
 'patient142',
 'patient143',
 'patient144',
 'patient145',
 'patient146',
 'patient147',
 'patient148',
 'patient149',
 'patient150']

In [38]:
# Read cfg file in each of the folders
patientDetails_test=[]
for folder in folders_test:
    cfg_file = os.path.join(ACDC_ORIG_DATA+'/testing', folder, 'Info.cfg')
    config_dict = {}
    with open(cfg_file, 'r') as file:
        lines = file.readlines()
        id = folder.split('patient')[-1]
        config_dict['patientID'] = id
        print(id)
        for line in lines:
            key, value = line.strip().split(': ')
            # Strip any extra spaces around the key and value
            key = key.strip()
            value = value.strip()
            # Add key-value pair to dictionary
            config_dict[key] = value
        print(config_dict)
    patientDetails_test.append(config_dict)

101
{'patientID': '101', 'ED': '1', 'ES': '14', 'Group': 'DCM', 'Height': '169.0', 'NbFrame': '30', 'Weight': '79.0'}
102
{'patientID': '102', 'ED': '1', 'ES': '13', 'Group': 'NOR', 'Height': '156.0', 'NbFrame': '30', 'Weight': '75.0'}
103
{'patientID': '103', 'ED': '1', 'ES': '11', 'Group': 'MINF', 'Height': '175.0', 'NbFrame': '30', 'Weight': '107.0'}
104
{'patientID': '104', 'ED': '1', 'ES': '11', 'Group': 'HCM', 'Height': '180.0', 'NbFrame': '30', 'Weight': '74.0'}
105
{'patientID': '105', 'ED': '1', 'ES': '10', 'Group': 'HCM', 'Height': '173.0', 'NbFrame': '30', 'Weight': '84.0'}
106
{'patientID': '106', 'ED': '1', 'ES': '13', 'Group': 'DCM', 'Height': '181.0', 'NbFrame': '30', 'Weight': '91.0'}
107
{'patientID': '107', 'ED': '1', 'ES': '10', 'Group': 'NOR', 'Height': '155.0', 'NbFrame': '30', 'Weight': '47.0'}
108
{'patientID': '108', 'ED': '1', 'ES': '9', 'Group': 'HCM', 'Height': '170.0', 'NbFrame': '30', 'Weight': '90.0'}
109
{'patientID': '109', 'ED': '1', 'ES': '10', 'Group'

In [39]:
patientDetails_test

[{'patientID': '101',
  'ED': '1',
  'ES': '14',
  'Group': 'DCM',
  'Height': '169.0',
  'NbFrame': '30',
  'Weight': '79.0'},
 {'patientID': '102',
  'ED': '1',
  'ES': '13',
  'Group': 'NOR',
  'Height': '156.0',
  'NbFrame': '30',
  'Weight': '75.0'},
 {'patientID': '103',
  'ED': '1',
  'ES': '11',
  'Group': 'MINF',
  'Height': '175.0',
  'NbFrame': '30',
  'Weight': '107.0'},
 {'patientID': '104',
  'ED': '1',
  'ES': '11',
  'Group': 'HCM',
  'Height': '180.0',
  'NbFrame': '30',
  'Weight': '74.0'},
 {'patientID': '105',
  'ED': '1',
  'ES': '10',
  'Group': 'HCM',
  'Height': '173.0',
  'NbFrame': '30',
  'Weight': '84.0'},
 {'patientID': '106',
  'ED': '1',
  'ES': '13',
  'Group': 'DCM',
  'Height': '181.0',
  'NbFrame': '30',
  'Weight': '91.0'},
 {'patientID': '107',
  'ED': '1',
  'ES': '10',
  'Group': 'NOR',
  'Height': '155.0',
  'NbFrame': '30',
  'Weight': '47.0'},
 {'patientID': '108',
  'ED': '1',
  'ES': '9',
  'Group': 'HCM',
  'Height': '170.0',
  'NbFrame': '3

In [40]:
def change_frame_cha(data):
    for item in data:
        if len(item['ED']) == 1:  # Check if 'ED' value is a single digit
            item['ED'] = item['ED'].zfill(2)  # Pad with leading zero to make it two digits
        if len(item['ES']) == 1:  # Check if 'ES' value is a single digit
            item['ES'] = item['ES'].zfill(2)  # Pad with leading zero to make it two digits
    return data

In [41]:
pat_changes_data = change_frame_cha(patientDetails_test)

In [42]:
pat_changes_data

[{'patientID': '101',
  'ED': '01',
  'ES': '14',
  'Group': 'DCM',
  'Height': '169.0',
  'NbFrame': '30',
  'Weight': '79.0'},
 {'patientID': '102',
  'ED': '01',
  'ES': '13',
  'Group': 'NOR',
  'Height': '156.0',
  'NbFrame': '30',
  'Weight': '75.0'},
 {'patientID': '103',
  'ED': '01',
  'ES': '11',
  'Group': 'MINF',
  'Height': '175.0',
  'NbFrame': '30',
  'Weight': '107.0'},
 {'patientID': '104',
  'ED': '01',
  'ES': '11',
  'Group': 'HCM',
  'Height': '180.0',
  'NbFrame': '30',
  'Weight': '74.0'},
 {'patientID': '105',
  'ED': '01',
  'ES': '10',
  'Group': 'HCM',
  'Height': '173.0',
  'NbFrame': '30',
  'Weight': '84.0'},
 {'patientID': '106',
  'ED': '01',
  'ES': '13',
  'Group': 'DCM',
  'Height': '181.0',
  'NbFrame': '30',
  'Weight': '91.0'},
 {'patientID': '107',
  'ED': '01',
  'ES': '10',
  'Group': 'NOR',
  'Height': '155.0',
  'NbFrame': '30',
  'Weight': '47.0'},
 {'patientID': '108',
  'ED': '01',
  'ES': '09',
  'Group': 'HCM',
  'Height': '170.0',
  'NbF

In [44]:
def test_save_one_frame(patientID,frameNum,savePath, size):
    imagePath = ACDC_ORIG_DATA+'/testing/' +'patient'+ patientID + '/patient'+ patientID + '_frame' + frameNum + '.nii.gz'
    maskPath = ACDC_ORIG_DATA+'/testing/' +'patient'+ patientID + '/patient'+ patientID + '_frame' + frameNum + '_gt.nii.gz'

    image = nib.load(imagePath)
    image_data = image.get_fdata()
    mask= nib.load(maskPath)
    mask_data = mask.get_fdata()

    unique_values = np.unique(mask_data)

    # Change the position from (height, width, depth) to (depth, height, width)
    image_data = np.moveaxis(image_data, -1, 0)
    mask_data = np.moveaxis(mask_data, -1, 0)

    # print(image_data_es.shape)
    # Resize and normalize all together and save it
    resizedimage=np.zeros((image_data.shape[0],size,size))
    resizedmask=np.zeros((mask_data.shape[0],size,size))

    for i in range(image_data.shape[0]):
        resized_image_slice = resize(image_data[i, :, :], (size, size), order=0, preserve_range=True) 
        resized_mask_slice = resize(mask_data[i, :, :], (size, size), order=0, preserve_range=True) 
        # # Normalize the image slice to range [-1, 1]
        min_val = np.min(resized_image_slice)
        max_val = np.max(resized_image_slice)

        normalized_image_slice = 2 * (resized_image_slice - min_val) / (max_val - min_val) - 1
        # print(normalized_image_slice.shape)
        resizedimage[i,:,:]=normalized_image_slice
        resizedmask[i,:,:]=resized_mask_slice
        unique_values_resized = np.unique(resized_mask_slice)
        # Print the unique values
        print(unique_values_resized)
    savefilePath= savePath #'./dataset/test/'+f'case_{patient["patientID"]}_volume_ES'
    print(savefilePath)
    np.savez_compressed(savefilePath, img=resizedimage, label=resizedmask)
    print("successfully saved the files for patientID: ", patientID)


In [46]:
for patient in patientDetails_test:
    #save ed frame
    edFilePath=ACDC_Save+'test/case_'+ patient["patientID"] +'_Volume_ED'
    fileNamesEd = test_save_one_frame(patient['patientID'],patient['ED'],edFilePath,224)
    # trainFileNames= np.concatenate((trainFileNames,fileNamesEd)) #= trainFileNames + fileNamesEd
    #save es frame
    esFilePath=ACDC_Save+'test/'+ 'case_'+ patient["patientID"] +'_Volume_ES'
    fileNamesEs=test_save_one_frame(patient['patientID'],patient['ES'],esFilePath,224)

[0. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
./DataProcessed/test/case_101_Volume_ED
successfully saved the files for patientID:  101
[0.]
[0. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
./DataProcessed/test/case_101_Volume_ES
successfully saved the files for patientID:  101
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0.]
./DataProcessed/test/case_102_Volume_ED
successfully saved the files for patientID:  102
[0.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0.]
./DataProcessed/test/case_102_Volume_ES
successfully saved the files for patientID:  102
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 2. 3.]
./DataProcessed/test/case_103_Volume_ED
successfully saved 

In [47]:
testFileNames_new= read_file_names(ACDC_Save+'test')
testFileNames_modified = np.char.replace(testFileNames_new, ACDC_Save+'test/', '')
np.savetxt(ACDC_Save+'lists_ACDC/test.txt', testFileNames_modified, fmt='%s')

In [48]:
testFileNames_modified

array(['case_149_Volume_ES.npz', 'case_149_Volume_ED.npz',
       'case_104_Volume_ED.npz', 'case_104_Volume_ES.npz',
       'case_113_Volume_ES.npz', 'case_113_Volume_ED.npz',
       'case_139_Volume_ED.npz', 'case_139_Volume_ES.npz',
       'case_117_Volume_ED.npz', 'case_117_Volume_ES.npz',
       'case_129_Volume_ES.npz', 'case_129_Volume_ED.npz',
       'case_110_Volume_ED.npz', 'case_110_Volume_ES.npz',
       'case_107_Volume_ES.npz', 'case_107_Volume_ED.npz',
       'case_103_Volume_ED.npz', 'case_103_Volume_ES.npz',
       'case_114_Volume_ES.npz', 'case_114_Volume_ED.npz',
       'case_140_Volume_ES.npz', 'case_140_Volume_ED.npz',
       'case_144_Volume_ED.npz', 'case_144_Volume_ES.npz',
       'case_127_Volume_ES.npz', 'case_127_Volume_ED.npz',
       'case_130_Volume_ED.npz', 'case_130_Volume_ES.npz',
       'case_109_Volume_ES.npz', 'case_109_Volume_ED.npz',
       'case_134_Volume_ES.npz', 'case_134_Volume_ED.npz',
       'case_123_Volume_ED.npz', 'case_123_Volume_ES.npz

### Getting z-spacing for test dataset

In [49]:
# getting z-spacing for test patients
z_spacing = {}
for patient in patientDetails_test:
    # Load the image
    image_path = ACDC_ORIG_DATA + '/testing' + '/patient' + patient['patientID'] + '/patient' + patient['patientID'] +"_frame"+ patient['ED'] +'.nii.gz'
    image = nib.load(image_path)
    image_data = image.get_fdata()
    # Get the z-spacing
    z_spacing['case_'+ patient["patientID"] +'_Volume_ED.npz'] = image.header['pixdim'][1:4] \
        #convert the value to 

    image_path_es = ACDC_ORIG_DATA + '/testing' + '/patient' + patient['patientID'] + '/patient' + patient['patientID'] +"_frame"+ patient['ES'] +'.nii.gz'
    image_es = nib.load(image_path_es)
    image_data_es = image_es.get_fdata()
    # Get the z-spacing
    z_spacing['case_'+ patient["patientID"] +'_Volume_ES.npz'] = image_es.header['pixdim'][1:4]

In [50]:
z_spacing

{'case_101_Volume_ED.npz': array([ 1.64062,  1.64062, 10.     ], dtype=float32),
 'case_101_Volume_ES.npz': array([ 1.64062,  1.64062, 10.     ], dtype=float32),
 'case_102_Volume_ED.npz': array([ 1.5625,  1.5625, 10.    ], dtype=float32),
 'case_102_Volume_ES.npz': array([ 1.5625,  1.5625, 10.    ], dtype=float32),
 'case_103_Volume_ED.npz': array([ 1.5625,  1.5625, 10.    ], dtype=float32),
 'case_103_Volume_ES.npz': array([ 1.5625,  1.5625, 10.    ], dtype=float32),
 'case_104_Volume_ED.npz': array([ 1.36719,  1.36719, 10.     ], dtype=float32),
 'case_104_Volume_ES.npz': array([ 1.36719,  1.36719, 10.     ], dtype=float32),
 'case_105_Volume_ED.npz': array([ 1.48438,  1.48438, 10.     ], dtype=float32),
 'case_105_Volume_ES.npz': array([ 1.48438,  1.48438, 10.     ], dtype=float32),
 'case_106_Volume_ED.npz': array([ 1.5625,  1.5625, 10.    ], dtype=float32),
 'case_106_Volume_ES.npz': array([ 1.5625,  1.5625, 10.    ], dtype=float32),
 'case_107_Volume_ED.npz': array([ 1.36719,  1

In [52]:
#save the z-spacing to a file
np.savez(ACDC_Save+'lists_ACDC/z_spacing_test', **z_spacing)

## Save Secret test data

In [53]:
ACDC_Secret_Test='/Users/poojakishore/Downloads/ACDC_SecretTest' #Add your secret test dataset path
folders_Secret_test = os.listdir(ACDC_Secret_Test)
print(folders_Secret_test)
folders_Secret_test.remove('.DS_Store')

['.DS_Store', 'patient159', 'patient166', 'patient161', 'patient168', 'patient157', 'patient151', 'patient169', 'patient156', 'patient160', 'patient158', 'patient167', 'patient174', 'patient173', 'patient172', 'patient175', 'patient162', 'patient165', 'patient153', 'patient154', 'patient155', 'patient152', 'patient164', 'patient163', 'patient170', 'patient171']


In [54]:
folders_Secret_test.sort()
folders_Secret_test

['patient151',
 'patient152',
 'patient153',
 'patient154',
 'patient155',
 'patient156',
 'patient157',
 'patient158',
 'patient159',
 'patient160',
 'patient161',
 'patient162',
 'patient163',
 'patient164',
 'patient165',
 'patient166',
 'patient167',
 'patient168',
 'patient169',
 'patient170',
 'patient171',
 'patient172',
 'patient173',
 'patient174',
 'patient175']

In [55]:
def secret_test_save_one_frame(patientID,savePath, size):
    print(patientID)
    imagePath = ACDC_Secret_Test +'/patient'+ patientID + '/patient'+ patientID + '_frame' + "01" + '.nii.gz'
    print(imagePath)

    image = nib.load(imagePath)
    image_data = image.get_fdata()
    print(image_data.shape)
    # print(image_data_es.shape)
    for j in range(image_data.shape[3]):
        img_data_vol = image_data[:,:,:,j]
        # Change the position from (height, width, depth) to (depth, height, width)
        img_data_vol = np.moveaxis(img_data_vol, -1, 0)

        # Resize and normalize all together and save it
        resizedimage_vol=np.zeros((img_data_vol.shape[0],size,size))
        print(resizedimage_vol.shape)

        for i in range(img_data_vol.shape[0]):
            resized_image_slice = resize(img_data_vol[i, :, :], (size, size), order=0, preserve_range=True) #anti_aliasing=True)
            # Normalize the image slice to range [-1, 1]
            min_val = np.min(resized_image_slice)
            max_val = np.max(resized_image_slice)

            normalized_image_slice = 2 * (resized_image_slice - min_val) / (max_val - min_val) - 1
            # print(normalized_image_slice.shape)
            resizedimage_vol[i,:,:]=normalized_image_slice
        savefilePath= savePath + "_" + str(j) #'./dataset/test/'+f'case_{patient["patientID"]}_volume_ES'
        print(savefilePath)
        np.savez_compressed(savefilePath, img=resizedimage_vol)
        print("successfully saved the files for patientID: ", patientID)


In [57]:

for patient in folders_Secret_test:
    #save ed frame
    print(patient)
    edFilePath=ACDC_Save+'secret/case_'+ patient +'_Volume'
    #get patient id from folder name
    patientID = patient.split('patient')[-1]
    fileNamesEd = secret_test_save_one_frame(patientID,edFilePath,224)

patient151
151
/Users/poojakishore/Downloads/ACDC_SecretTest/patient151/patient151_frame01.nii.gz
(216, 256, 13, 25)
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_0
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_1
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_2
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_3
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_4
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_5
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_6
successfully saved the files for patientID:  151
(13, 224, 224)
./DataProcessed/secret/case_patient151_Volume_7
successfully saved the files for pat

In [58]:
testFileNames_new= read_file_names(ACDC_Save+'secret')
testFileNames_modified = np.char.replace(testFileNames_new, ACDC_Save+'secret/', '')
np.savetxt(ACDC_Save+'lists_ACDC/secrettest.txt', testFileNames_modified, fmt='%s')

In [59]:
testFileNames_modified

array(['case_patient164_Volume_12.npz', 'case_patient170_Volume_17.npz',
       'case_patient159_Volume_13.npz', 'case_patient158_Volume_21.npz',
       'case_patient171_Volume_25.npz', 'case_patient159_Volume_3.npz',
       'case_patient165_Volume_20.npz', 'case_patient171_Volume_19.npz',
       'case_patient166_Volume_19.npz', 'case_patient172_Volume_20.npz',
       'case_patient169_Volume_6.npz', 'case_patient166_Volume_25.npz',
       'case_patient167_Volume_17.npz', 'case_patient173_Volume_12.npz',
       'case_patient160_Volume_13.npz', 'case_patient174_Volume_16.npz',
       'case_patient175_Volume_24.npz', 'case_patient161_Volume_21.npz',
       'case_patient175_Volume_18.npz', 'case_patient158_Volume_4.npz',
       'case_patient162_Volume_18.npz', 'case_patient168_Volume_1.npz',
       'case_patient162_Volume_24.npz', 'case_patient163_Volume_16.npz',
       'case_patient163_Volume_17.npz', 'case_patient168_Volume_0.npz',
       'case_patient162_Volume_19.npz', 'case_patient158

In [ ]:
# getting z-spacing for test patients
z_spacing_secret = {}
for patient in folders_Secret_test:
    # Load the image
    patientID = patient.split('patient')[-1]
    imagePath = ACDC_Secret_Test +'/'+ patient + '/'+ patient + '_frame' + "01" + '.nii.gz'
    image = nib.load(imagePath)
    image_data = image.get_fdata()
    # Get the z-spacing
    # print(image.header['pixdim'])
    for j in range(image_data.shape[3]):
        z_spacing_secret['case_'+patientID +'_Volume_'+str(j)+'.npz'] = image.header['pixdim'][1:4] 
        #convert the value to 

In [ ]:
#save the z-spacing to a file
np.savez(ACDC_Save+'lists_ACDC/z_spacing_secrettest', **z_spacing_secret)